In [19]:
!pip install langchain


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [21]:
from langchain.prompts import PromptTemplate

In [23]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
 )
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [25]:
prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

In [27]:
from langchain_core.prompts import ChatPromptTemplate

In [29]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}"),
    ("human", "Hello, how are you doing?"),
    ("ai", "I am doing well, thanks!"),
    ("human", "{user_input}") ]
)
messages = chat_template.format_messages(name="Bob", user_input="What is your name?")

In [31]:
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content='I am doing well, thanks!'),
 HumanMessage(content='What is your name?')]

In [33]:
!pip install langchain_openai


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip


In addition to using the 2-tupple representation of (type, content) used above, you could pass in an instance of MessagePromptTemplate or BaseMessage

In [35]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=("You are a helpful assistant that re-writes the user's text to sound more upbeat.")), HumanMessagePromptTemplate.from_template("{text}")
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


# Few-shot Prompte Templates

## 1 - Using an Example Set

In [36]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


## create the sample set

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

# create a formatter for the few-shot examples
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question {question}\n{answer}")
print(example_prompt.format(**examples[0]))

Question Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [37]:
# feed examples and formatter to FewShowPromptTemplate

prompt = FewShotPromptTemplate(
    examples = examples, example_prompt = example_prompt, suffix="Question: {input}", input_variables=["input"],
)
print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Wash

In [38]:
!pip install chromadb


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Using an Example Selector

In [39]:
## feed examples into an ExampleSelector object (instance)
## We will use the SemanticSimilarityExampleSelector which uses a model to calculate the semantic similarity  between the input and examples.



from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity
    OllamaEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce
    k=1
)

# Select the most similar example to the input
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input {question}")
for example in selected_examples:
  print("\n")
  for k, v in example.items():
    print(f"{k}: {v}")

Examples most similar to the input Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

question: Who lived longer, Muhammad Ali or Alan Turing?


In [42]:
# feed example selector into FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    suffix = "Question: {input}",
    input_variables=["input"]
)
print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: Who was the father of Mary Ball Washington?


# Few-shot examples for Chat Models

In [43]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [44]:
examples = [ {"input": "2+2", "output": "4"}, { "input": "2+3", "output": "5" } ] 

In [45]:
# assemble them into the few-shot prompt template

In [46]:
# This is a prompt template used to format each individual example.

In [48]:
example_prompt = ChatPromptTemplate.from_messages(
    [ ("human", "{input}"), ("ai", "{output}") ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)
print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [49]:
# finally assemble your final prompt and use it with a model

In [52]:
final_prompt = ChatPromptTemplate.from_messages(
    [ ("system", "You are a wondrous wizard of math."), few_shot_prompt, ( "human", "{input}" )  ]
)

In [57]:
from langchain_community.chat_models import ChatOllama
chain = final_prompt | ChatOllama()
chain.invoke({ "input": "What's the square of a triangle?" })

AIMessage(content='I\'m happy to help you with your question, but I must inform you that the concept of a "square of a triangle" is not a well-defined mathematical concept. Triangles are geometric shapes with three sides and three angles, and they do not have a squared form. Therefore, the square of a triangle does not exist in mathematics. Is there anything else I can help you with?')

# Dynamic few-shot Prompting

In [58]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma

In [59]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

In [60]:
to_vectorize = [ " ".join(example.values()) for example in examples ]
embeddings = OllamaEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [62]:
vectorstore

In [66]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore = vectorstore,
    k=2
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({ "input": "horse" })

[{'input': '2+2', 'output': '4'}, {'input': '2+3', 'output': '5'}]